In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns


* 전체 요약 순서 -
 
1. 데이터 불러오기

2. EDA (구조 확인, 통계, 시각화)

3. 결측치 처리

4. 이상치 처리

5. 인코딩

6. 스케일링

7. 학습/테스트 분리

8. 모델 학습

9. 예측 & 평가

10. (선택) 성능 개선

In [4]:
# 모든 분석은 데이터를 불러오는 것부터 시작하기 때문이야. 데이터 없이는 어떤 작업도 못 해!
df = pd.read_csv("../data/penguins_size.csv")
df

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


In [5]:
# 데이터를 읽고 나면 구조를 바로 파악해야 데이터 전처리를 어떻게 할지 감을 잡을 수 있어.
df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [6]:
# 데이터를 분석하거나 전처리할 때 원본 데이터를 보존하고 싶을 때 사용해.
# 예를 들어 이후에 df를 수정하더라도 df_raw는 처음 읽은 상태 그대로 남아 있어, 
# 필요하면 다시 참고할 수 있어.
df_raw = df.copy()

In [7]:
# 데이터 정제 전에 결측치나 데이터 타입 문제를 사전에 파악해야 올바른 처리를 할 수 있어.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   culmen_length_mm   342 non-null    float64
 3   culmen_depth_mm    342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                334 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [8]:
# 데이터 분포나 이상치 여부, 전반적 규모 등을 확인하고 전처리/스케일링 전략을 세우는 데 필수야.
df.describe()

,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
count,342.000000,342.000000,342.000000,342.000000
mean,43.921930,17.151170,200.915205,4201.754386
std,5.459584,1.974793,14.061714,801.954536
min,32.100000,13.100000,172.000000,2700.000000
25%,39.225000,15.600000,190.000000,3550.000000
50%,44.450000,17.300000,197.000000,4050.000000
75%,48.500000,18.700000,213.000000,4750.000000
max,59.600000,21.500000,231.000000,6300.000000


In [9]:
# 각 컬럼에 비어 있는 값(결측치)이 몇 개인지 알려주는 코드예요.
df.isnull().sum()

species               0
island                0
culmen_length_mm      2
culmen_depth_mm       2
flipper_length_mm     2
body_mass_g           2
sex                  10
dtype: int64

In [10]:
df["sex"].unique()

array(['MALE', 'FEMALE', nan, '.'], dtype=object)

In [11]:
# value_counts(): "sex"라는 컬럼 안에 있는 
# "MALE","FEMALE" values의 개수를 파악할 때 사용하는 함수.
df["sex"].value_counts()

sex
MALE      168
FEMALE    165
.           1
Name: count, dtype: int64

In [ ]:
# 결측치 처리과정 숫자들은 평균을 내서 결측치를 채워주고 문자열은 비우거나 많은 값으로 채운다.
df["culmen_length_mm"] = df["culmen_length_mm"].fillna(df["culmen_length_mm"].mean())
df["culmen_depth_mm"] = df["culmen_depth_mm"].fillna(df["culmen_depth_mm"].mean())
df["flipper_length_mm"] = df["flipper_length_mm"].fillna(df["flipper_length_mm"].mean())
df["body_mass_g"] = df["body_mass_g"].fillna(df["body_mass_g"].mean())
df["sex"] = df["sex"].fillna("MALE")

In [19]:
df.isna().sum()

species              0
island               0
culmen_length_mm     0
culmen_depth_mm      0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64

In [ ]:
from sklearn.preprocessing import L